# BAMBOO: Binary descriptor based on AsymMetric pairwise BOOsting

In this notebook we include the implementation of the BAMBOO descriptor to provide a compressed representation of probe requests.

## Libraries and Configurations

Logger

In [2]:
import logging
from rich.logging import RichHandler

logging.getLogger("scapy.runtime").setLevel(logging.CRITICAL)

FORMAT = "%(message)s"
logging.basicConfig(
    level="NOTSET",
    format=FORMAT,
    datefmt="[%X]",
    handlers=[RichHandler(rich_tracebacks=True)],
)

log = logging.getLogger("rich")

log.setLevel("CRITICAL")

Import configuration files

In [3]:
from configparser import ConfigParser

config = ConfigParser()
config.read("../config.ini")

['../config.ini']

Import **data libraries**

In [4]:
import pandas as pd

Import **other libraries**

In [5]:
from rich.progress import Progress
from rich import traceback

traceback.install()

from tqdm.notebook import tqdm

In [6]:
import numpy as np
import math

Fancy print

In [7]:
class color:
    PURPLE = "\033[95m"
    CYAN = "\033[96m"
    DARKCYAN = "\033[36m"
    BLUE = "\033[94m"
    GREEN = "\033[92m"
    YELLOW = "\033[93m"
    RED = "\033[91m"
    BOLD = "\033[1m"
    UNDERLINE = "\033[4m"
    END = "\033[0m"

In [8]:
def bold_text(text: str) -> str:
    return str(color.BOLD + str(text) + color.END)

## Import Data

Importing **concatenated columns** and **pairs** datasets

In [9]:
pairs_df = pd.read_csv("../../data/interim/pairs_df.csv", index_col=0)

In [10]:
pairs_df

,Item 1,Item 2,Equality
0,0,1,1
1,0,2,1
2,0,3,1
3,0,4,1
4,0,5,1
...,...,...,...
54280,326,328,1
54281,326,329,1
54282,327,328,1
54283,327,329,1


In [11]:
strings_df = pd.read_csv("../../data/interim/string_df.csv", index_col=0)
strings_df = strings_df.rename(columns={strings_df.columns[0]: "Probes"})

In [12]:
strings_df

,Probes
0,0000000000000100000000100000010000001011000101...
1,0000000000000100000000100000010000001011000101...
2,0000000000000100000000100000010000001011000101...
3,0000000000000100000000100000010000001011000101...
4,0000000000000100000000100000010000001011000101...
...,...
325,0000000000000100000000100000010000001011000101...
326,0000000000000100000000100000010000001011000101...
327,0000000000000100000000100000010000001011000101...
328,0000000000000100000000100000010000001011000101...


Importing bitmask **filters**

In [13]:
filters_df = pd.read_csv("../../data/filters/bitmasks.csv", index_col=0)

In [14]:
print("Number of filters in the dataset: " + bold_text(filters_df.shape[0]))

Number of filters in the dataset: 16


Select only filter corresponding to single columns (drop combinations).

In [15]:
filters_df = filters_df.head(16).reset_index()

Getting actual bitmask filters' column

In [16]:
filters = filters_df["Bitmask"]

Function to associate a **set of thresholds to each filter**, depending on the number of `1`.

In [17]:
def generate_thresholds(bitmasks):
    """
    Generate thresholds for each bitmask in a set.

    Parameters:
        bitmasks (set): A set containing the bitmasks.

    Returns:
        list: A list of tuples where each tuple contains a list of bitmasks sharing the same threshold list, and their corresponding threshold list.
    """
    threshold_dict = {}
    for bitmask in bitmasks:
        threshold = bitmask.count("1")
        if threshold in threshold_dict:
            threshold_dict[threshold].append(bitmask)
        else:
            threshold_dict[threshold] = [bitmask]

    return [
        (bitmasks, list(range(threshold + 1)))
        for threshold, bitmasks in threshold_dict.items()
    ]

**Generating thresholds** from bitmask filters

In [18]:
thresholds_list = generate_thresholds(filters)

In [19]:
def generate_thresholds_df(bitmasks: set) -> pd.DataFrame:
    """
    Generate a DataFrame with filters and corresponding thresholds.

    Parameters:
        bitmasks (set): A set containing the bitmasks.

    Returns:
        pandas.DataFrame: DataFrame with two columns - 'filters' and 'thresholds'.
    """
    data = []
    for bitmask in bitmasks:
        threshold = bitmask.count("1")
        thresholds = list(range(threshold + 1))  # Include 0 to the count of '1's
        data.append((bitmask, thresholds))

    return pd.DataFrame(data, columns=["filters", "thresholds"])

In [20]:
filter_threshold_df = generate_thresholds_df(filters)

In [21]:
filter_threshold_df

,filters,thresholds
0,1111111100000000000000000000000000000000000000...,"[0, 1, 2, 3, 4, 5, 6, 7, 8]"
1,0000000011111111000000000000000000000000000000...,"[0, 1, 2, 3, 4, 5, 6, 7, 8]"
2,0000000000000000111111111111111111111111111111...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
3,0000000000000000000000000000000000000000000000...,"[0, 1, 2, 3, 4, 5, 6, 7, 8]"
4,0000000000000000000000000000000000000000000000...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
5,0000000000000000000000000000000000000000000000...,"[0, 1, 2, 3, 4, 5, 6, 7, 8]"
6,0000000000000000000000000000000000000000000000...,"[0, 1, 2, 3, 4, 5, 6, 7, 8]"
7,0000000000000000000000000000000000000000000000...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
8,0000000000000000000000000000000000000000000000...,"[0, 1, 2, 3, 4, 5, 6, 7, 8]"
9,0000000000000000000000000000000000000000000000...,"[0, 1, 2, 3, 4, 5, 6, 7, 8]"


In [41]:
for index, row in filter_threshold_df.iterrows():
    filter_value = row["filters"]
    thresholds = row["thresholds"]

    print(f"For filter {filter_value}:")
    for threshold in thresholds:
        print(f"Threshold: {threshold}")

For filter 11111111000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

Deleting all the last thresholds.

In [23]:
for i in range(len(thresholds_list)):
    del thresholds_list[i][1][-1:-3]

In [24]:
print("Number of threshold sets: " + bold_text(len(thresholds_list)))

Number of threshold sets: 7


### Functions

The **bitwise AND** function performs said operation on 2 binary strings

In [25]:
def bitwise_and(bit_str1, bit_str2):
    # Convert bit strings to integers
    int1 = int(bit_str1, 2)
    int2 = int(bit_str2, 2)

    # Perform bitwise AND operation
    result = int1 & int2

    # Convert result back to binary string
    result_str = bin(result)[2:]  # [2:] to remove '0b' prefix

    # Return result
    return result_str.zfill(max(len(bit_str1), len(bit_str2)))

The **sum filter** takes as input a (binary) string and sums the values

In [26]:
def sumFilter(bitwise_and: str) -> int:
    sum = 0
    for i in bitwise_and:
        sum += int(i)
    return sum

**Sign function** returns -1 if negative value

In [27]:
def sign(number: int) -> int:
    if number < 0:
        return -1
    elif number >= 0:
        return 1

The **weak classifier** filters a couple of tuples, and given a threshold it, returns +1 or -1

In [28]:
def weak_classifier(pair: tuple, threshold: int, filter: str) -> int:
    log.debug(pair, threshold, filter)
    filtered1 = sumFilter(bitwise_and(pair[0], filter))
    filtered2 = sumFilter(bitwise_and(pair[1], filter))
    return sign((filtered1 - threshold) * (filtered2 - threshold))

Implementation of the **Dirach delta** function

In [29]:
def delta(prediction: int, ground_truth: int) -> int:
    if prediction != ground_truth:
        return 1
    else:
        return 0

The **get error** function calculates the weighted value of the filter, given the prevision and the ground truth

In [30]:
def get_error(weigth: float, prediction: int, ground_truth: int) -> float:
    error = weigth * delta(prediction, ground_truth)

    log.debug(f"Weigth {weigth}, Prediction {prediction}, Ground Truth {ground_truth}")

    return error

### BAMBOO

Input:
- Ground truth relationships $\langle x_{a(n)}, x_{b(n)}; y_n\rangle$
  - $n=1,..,N$
  - $y_n \in \{+1, -1\}$
- A set of filters $\mathcal{H} = \{h_1 , ..., h_F\}$
- A set of binarization thresholds $\mathcal{T} = \{t_1 , ..., t_T\}$

Output:
- A set of $M<F$ filters $[h_{i(1)}, ..., h_{i(M)}]$
- Corresponding set of binarization thresholds $[t_{j(1)}, ..., t_{j(M)}]$

Define **BAMBOO input**

In [31]:
# Input
dataset = strings_df.copy()
# pairs_index = pairs_df.copy()
pairs_index = pairs_df.head(500)
filters = thresholds_list
M = 10

# Initial weights
weights = np.ones(len(pairs_index)) / len(pairs_index)

# Errors per iteration
errors = {}

Algorithm implementation

In [32]:
for m in tqdm(range(M), desc="Iterations"):  # iterations
    best_filter = None
    best_threshold = None
    for filters_entry in tqdm(filters, desc="Filters"):  # for each filter
        filters_list, threshold_list = filters_entry

        for filter, thresholds in zip(
            filters_list, [threshold_list] * len(filters_list)
        ):
            for threshold in thresholds:  # for each threshold
                error = 0
                for pair in range(len(pairs_index)):  # for each pair
                    prediction = weak_classifier(
                        tuple(dataset.iloc[pairs_index.iloc[pair, 0:2], 0]),
                        threshold,
                        filter,
                    )
                    error += get_error(
                        weights[pair], prediction, pairs_index.iloc[pair, 2]
                    )
                errors[(filter, threshold)] = error
    best_filter, best_threshold = min(errors, key=lambda k: abs(errors[k]))

    print("Best Filter:", best_filter)
    print("Best Threshold:", best_threshold)

    min_error = errors[(best_filter, best_threshold)]
    confidence = math.log(
        (1 - min_error) / min_error
    )  # confidence of the weak classifier
    print("Min error", min_error)
    print("Confidence:", confidence)

    # Asymmetric Weight Update
    for pair in range(len(pairs_index)):

        # print(
        #     dataset[pair][2],
        #     weak_classifier(dataset[pair][0:2], best_threshold, best_filter),
        # )

        if pairs_index.iloc[pair, 2] == +1:
            if (
                weak_classifier(
                    tuple(dataset.iloc[pairs_index.iloc[pair, 0:2], 0]),
                    best_threshold,
                    best_filter,
                )
                != pairs_index.iloc[pair, 2]
            ):
                weights[pair] = weights[pair] * math.exp(confidence)

    for pair in range(len(pairs_index)):
        if pairs_index.iloc[pair, 2] == +1:
            weights[pair] = weights[pair] / sum(
                weights[pair]
                for pair in range(len(pairs_index))
                if pairs_index.iloc[pair, 2] == +1
            )

Iterations:   0%|          | 0/10 [00:00<?, ?it/s]

Filters:   0%|          | 0/7 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:13                                                                                   │
│                                                                                                  │
│   10 │   │   │   for threshold in thresholds:  # for each threshold                              │
│   11 │   │   │   │   error = 0                                                                   │
│   12 │   │   │   │   for pair in range(len(pairs_index)):  # for each pair                       │
│ ❱ 13 │   │   │   │   │   prediction = weak_classifier(                                           │
│   14 │   │   │   │   │   │   tuple(dataset.iloc[pairs_index.iloc[pair, 0:2], 0]),                │
│   15 │   │   │   │   │   │   threshold,                                                          │
│   16 │   │   │   │   │   │   filter,                                                             │
│                                                                                                  │
│ in weak_classifier:3                                                                             │
│                                                                                                  │
│   1 def weak_classifier(pair: tuple, threshold: int, filter: str) -> int:                        │
│   2 │   log.debug(pair, threshold, filter)                                                       │
│ ❱ 3 │   filtered1 = sumFilter(bitwise_and(pair[0], filter))                                      │
│   4 │   filtered2 = sumFilter(bitwise_and(pair[1], filter))                                      │
│   5 │   return sign((filtered1 - threshold) * (filtered2 - threshold))                           │
│   6                                                                                              │
│                                                                                                  │
│ in sumFilter:4                                                                                   │
│                                                                                                  │
│   1 def sumFilter(bitwise_and: str) -> int:                                                      │
│   2 │   sum = 0                                                                                  │
│   3 │   for i in bitwise_and:                                                                    │
│ ❱ 4 │   │   sum += int(i)                                                                        │
│   5 │   return sum                                                                               │
│   6                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

In [ ]:
print("Best Filter:", best_filter)
print("Best Threshold:", best_threshold)
print("Min error", min_error)

Best Filter: 000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111